# Lecture 2

Basic GPU programming

In [ ]:
print('Hello')

Hello


In [1]:
# Step 1: Write CUDA code to a .cu file
%%writefile vector_add.cu

#include <cuda_runtime.h>
#include <stdio.h>

__global__ void vector_add(float *a, float *b, float *c, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) c[i] = a[i] + b[i];
}

int main() {
    int N = 1024;
    float *a, *b, *c;
    float *d_a, *d_b, *d_c;
    int size = N * sizeof(float);

    // Allocate host memory
    a = (float*)malloc(size);
    b = (float*)malloc(size);
    c = (float*)malloc(size);

    // Initialize host arrays
    for (int i = 0; i < N; i++) a[i] = b[i] = 1.0f;

    // Allocate device memory
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);

    // Copy data to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel
    vector_add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c, N);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print result
    for (int i = 0; i < 10; i++) printf("%f ", c[i]);
    printf("\n");

    // Free memory
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
    free(a); free(b); free(c);

    return 0;
}

Writing vector_add.cu


In [2]:
# Step 2: Compile the CUDA code
!nvcc -o vector_add vector_add.cu

In [3]:
# Step 3: Run the compiled program
!./vector_add

2.000000 2.000000 2.000000 2.000000 2.000000 2.000000 2.000000 2.000000 2.000000 2.000000 
